In [2]:
import yfinance as yf
import tensorflow as tf
import numpy as np
import pandas as pd
import requests
from transformers import pipeline
import os
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Flatten, Concatenate
import tweepy

/Users/cr/Desktop/StockBot/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
TOP_25_STOCKS = ['AAPL', 'MSFT', 'AMZN', 'NVDA', 'GOOGL', 
                 'TSLA', 'GOOG', 'BRK-B', 'META', 'UNH', 
                 'XOM', 'LLY', 'JPM', 'JNJ', 'V', 
                 'PG', 'MA', 'AVGO', 'HD', 'CVX', 
                 'MRK', 'ABBV', 'COST', 'PEP', 'ADBE']

#Encoding tickers
label_encoder = LabelEncoder()
TICKERS_ENCODED = label_encoder.fit_transform(TOP_25_STOCKS)
print(TICKERS_ENCODED)
TICKERS_ONEHOT = to_categorical(TICKERS_ENCODED)
print(TICKERS_ONEHOT)

In [12]:
#downloading stock data

directory = 'TrainStockData'

for stock in TOP_25_STOCKS:
    data = yf.download(stock, start="2012-01-01", end="2024-01-01")
    data = data.drop(columns=['Open', 'High', 'Low', 'Adj Close'])
    data.insert(0, 'Ticker', stock, True)
    sentiments = []
    data['Date'] = data.index
    data['index'] = range(len(data))
    data.set_index('index', inplace=True)
    print(data.index)
    # for date in data.index:
    #     year = int(date.split('-')[0])
    #     month = int(date.split('-')[1])
    #     day = int(date.split('-')[2])
    
    #     sumSentiments = 0
    #     numSentiments = 0
    #     print(date)
        
    #     avgSentiments = None
    #     if numSentiments != 0:
    #         avgSentiments = sumSentiments/numSentiments
        
    data.to_csv(os.path.join(directory, f'Train_{stock}.csv'))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
Index([   0,    1,    2,    3,

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       2448, 2449, 2450, 2451, 2452, 2453, 2454, 2455, 2456, 2457],
      dtype='int64', name='index', length=2458)
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       2913, 2914, 2915, 2916, 2917, 2918, 2919, 2920, 2921, 2922],
      dtype='int64', name='index', length=2923)
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
Index([   0,    1,    2,    3,


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
Index([   0,    1,    2,    3,

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       2758, 2759, 2760, 2761, 2762, 2763, 2764, 2765, 2766, 2767],
      dtype='int64', name='index', length=2768)
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017],
      dtype='int64', name='index', length=3018)
